In [1]:
import numpy as np
from datetime import datetime, timedelta
import random
import math

HV = timedelta(days=10**6)

# Configuración inicial
NC = 1  # número de cajas
NB = 2  # número de baristas
TF = timedelta(days=6, hours=23, minutes=59);  # tiempo final de simulación
TI = timedelta(hours=00, minutes=00);  # tiempo inicial de simulación

# Variables de estado
T = TI;
TPLL = TI; # primera llegada
TPSC = [HV] * NC  # tiempos de fin de atención en caja
TPSB = [HV] * NB  # tiempos de fin de atención en barista
NSC = 0  # número de clientes en cola de caja
NSB = 0  # número de pedidos esperando
NT = 0  # número de clientes atendidos
NE = 0  # número de pedidos atendidos

# Fdps
# variables TAC
mu_TAC = 1
sigma_TAC = 1.1299538126456397

# variables TAB
mu_TAB = 2
sigma_TAB = 1.4684641705996866

# variables IA
beta_IA = 1.067608417909708
mu_IA = 0.43830138260945994
sigma_IA = 6.146166055817218 #scale

#Half-logistic
def half_logistic(R, mu, sigma):
    return -sigma * np.log((1 - R) / (R + 1)) + mu

#Truncexpon
def trunc_expon(r, beta, scale, loc):
  inversa = -np.log(1 - r * (1 - np.exp(-beta)))
  return scale * inversa + loc

def tac(mu,sigma):
  r = np.random.random();
  return timedelta(minutes=half_logistic(r, mu, sigma));

def tab(mu,sigma):
  r = np.random.random();
  return timedelta(minutes=half_logistic(r, mu, sigma));

def ia():
  r = np.random.random();
  return timedelta(minutes=trunc_expon(r, beta_IA, sigma_IA, mu_IA));

# Estadísticas, timedelta = (00:00:00)
SPS = timedelta()
STOC = [timedelta()] * NC
STOB = [timedelta()] * NB
ITOC = [TI] * NC
ITOB = [TI] * NB
PTOB = [timedelta()] * NB
PTOC = [timedelta()] * NC
PTOTotal = 0;


def buscar_tpsc_menor():
    return TPSC.index(min(TPSC))

def buscar_tpsc_libre_o_menor():
    if HV in TPSC:
       return TPSC.index(HV)
    return TPSC.index(min(TPSC))

def buscar_tpsb_menor():
    return TPSB.index(min(TPSB))

def buscar_tpsb_libre_o_menor():
    if HV in TPSB:
       return TPSB.index(HV)
    return TPSB.index(min(TPSB))

# Simulación
while (
    TPLL != HV or
    any(t < HV for t in TPSC) or
    any(t < HV for t in TPSB)
):
  i = buscar_tpsc_menor();
  j = buscar_tpsb_menor();
  if TPLL != HV and TPLL <= TPSC[i] and TPLL <= TPSB[j]:
    #Llegada
    SPS += (TPLL-T)*(NSC+NSB);
    T = TPLL;
    NSC += 1;
    NT += 1;
    IA = ia();
    if T < TF:
      IA = ia()
      TPLL = T + IA
    else:
      TPLL = HV
    if(NSC <= NC):
      i = buscar_tpsc_libre_o_menor()
      STOC[i] += (T - ITOC[i]);
      TAC = tac(mu_TAC, sigma_TAC);
      TPSC[i] = T + TAC;
  elif NSC > 0 and TPSC[i] != HV and TPSC[i] < TPLL and TPSC[i] <= TPSB[j]:
    #Caja
    SPS += (TPSC[i] - T)*(NSC + NSB);
    T = TPSC[i];
    NSC -= 1;
    if(NSC>=NC):
      TAC = tac(mu_TAC, sigma_TAC);
      TPSC[i] = T + TAC;
    else:
      ITOC[i] = T;
      TPSC[i] = HV;
    r = np.random.random();
    if(r <= 0.55):
      NSB += 1;
      NE += 1;
      if(NSB<=NB):
        j = buscar_tpsb_libre_o_menor();
        STOB[j] += (T - ITOB[j]);
        TAB = tab(mu_TAB, sigma_TAB);
        TPSB[j] = T + TAB;
  elif NSB > 0 and TPSB[j] != HV and TPSB[j] <= TPLL and TPSB[j] <= TPSC[i]:
    #Barista
    SPS += (TPSB[j] - T)*(NSC+NSB);
    T = TPSB[j];
    NSB -= 1;
    if(NSB>=NB):
      TAB = tab(mu_TAB, sigma_TAB);
      TPSB[j] = T + TAB;
    else:
      ITOB[j] = T;
      TPSB[j] = HV;

# Estadísticas finales
for i in range(NC):
  PTOC[i] = STOC[i]*100/(TF-TI)

for j in range(NB):
  PTOB[j] = STOB[j]*100/(TF-TI)

PPS = SPS/NT

print(f"N° de cajas: {NC}");
print(f"N° de baristas: {NB}");
print(f"N° de clientes elaborados: {NE}");
print(f"N° de clientes atendidos: {NT}");
print(f"Promedio de permanencia en el sistema: {PPS}");

PTOTotalC = 0;
PTOTotalB = 0;

for i in range(NC):
  print(f"Porcentaje Tiempo ocioso Caja N°{i} : %{round(PTOC[i], 2)}");
  PTOTotalC += PTOC[i];

for j in range(NB):
  print(f"Porcentaje Tiempo ocioso Barista N°{j}: %{round(PTOB[j], 2)}");
  PTOTotalB += PTOB[j];

print(f"Promedio de porcentaje de tiempo ocioso total: %{round((PTOTotalC+PTOTotalB)/(NT), 2)}")
print(f"Promedio de porcentaje de tiempo ocioso caja: %{round(PTOTotalC/NC, 2)}")
print(f"Promedio de porcentaje de tiempo ocioso barista: %{round(PTOTotalB/NB, 2)}")

N° de cajas: 1
N° de baristas: 2
N° de clientes elaborados: 1724
N° de clientes atendidos: 3169
Promedio de permanencia en el sistema: 0:07:47.394925
Porcentaje Tiempo ocioso Caja N°0 : %19.66
Porcentaje Tiempo ocioso Barista N°0: %52.4
Porcentaje Tiempo ocioso Barista N°1: %78.9
Promedio de porcentaje de tiempo ocioso total: %0.05
Promedio de porcentaje de tiempo ocioso caja: %19.66
Promedio de porcentaje de tiempo ocioso barista: %65.65
